In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
SCHEDULE_URL = 'https://statsapi.mlb.com/api/v1/schedule'
TEAMS_URL = 'https://statsapi.mlb.com/api/v1/teams'
params = {'sportId': 1, 'season': 2023}


In [3]:
teams_resp = requests.get(TEAMS_URL, params | {'hydrate': 'standings'}).json()
teams_df = pd.json_normalize(teams_resp['teams'])

teams_df

,allStarStatus,id,name,link,season,teamCode,fileCode,abbreviation,teamName,locationName,...,record.eliminationNumberDivision,record.eliminationNumberConference,record.wildCardEliminationNumber,record.wins,record.losses,record.runDifferential,record.winningPercentage,record.wildCardLeader,record.magicNumber,record.clinchIndicator
0,N,133,Oakland Athletics,/api/v1/teams/133,2023,oak,oak,OAK,Athletics,Oakland,...,E,63,E,46,99,-295,.317,NaN,NaN,NaN
1,N,134,Pittsburgh Pirates,/api/v1/teams/134,2023,pit,pit,PIT,Pirates,Pittsburgh,...,4,84,9,67,78,-90,.462,NaN,NaN,NaN
2,N,135,San Diego Padres,/api/v1/teams/135,2023,sdn,sd,SD,Padres,San Diego,...,E,84,9,68,78,55,.466,NaN,NaN,NaN
3,N,136,Seattle Mariners,/api/v1/teams/136,2023,sea,sea,SEA,Mariners,Seattle,...,16,97,-,80,65,103,.552,True,NaN,NaN
4,N,137,San Francisco Giants,/api/v1/teams/137,2023,sfn,sf,SF,Giants,San Francisco,...,4,91,16,74,71,-8,.510,NaN,NaN,NaN
5,N,138,St. Louis Cardinals,/api/v1/teams/138,2023,sln,stl,STL,Cardinals,St. Louis,...,1,81,6,64,81,-74,.441,NaN,NaN,NaN
6,N,139,Tampa Bay Rays,/api/v1/teams/139,2023,tba,tb,TB,Rays,St. Petersburg,...,15,105,-,89,57,197,.610,True,NaN,NaN
7,N,140,Texas Rangers,/api/v1/teams/140,2023,tex,tex,TEX,Rangers,Arlington,...,17,98,-,80,64,151,.556,True,NaN,NaN
8,N,141,Toronto Blue Jays,/api/v1/teams/141,2023,tor,tor,TOR,Blue Jays,Toronto,...,7,97,-,80,65,72,.552,True,NaN,NaN
9,N,142,Minnesota Twins,/api/v1/teams/142,2023,min,min,MIN,Twins,Minneapolis,...,-,93,-,76,69,72,.524,NaN,10,NaN


In [4]:
ratings = teams_df.set_index('abbreviation')['record.winningPercentage']
ratings

abbreviation
OAK    .317
PIT    .462
SD     .466
SEA    .552
SF     .510
STL    .441
TB     .610
TEX    .556
TOR    .552
MIN    .524
PHI    .545
ATL    .655
CWS    .386
MIA    .510
NYY    .503
MIL    .563
LAA    .466
AZ     .521
BAL    .632
BOS    .503
CHC    .534
CIN    .514
CLE    .473
COL    .361
DET    .458
HOU    .562
KC     .308
LAD    .611
WSH    .448
NYM    .458
Name: record.winningPercentage, dtype: object

In [5]:
schedule_data = requests.get(SCHEDULE_URL, params | {'hydrate': 'team'}).json()
all_gms = pd.json_normalize(schedule_data['dates'], record_path=['games']).set_index('gamePk')
reg = all_gms.query('gameType=="R"')

# Remove the stubs of games that were rescheduled or suspended
remove = pd.concat([reg['resumeDate'].dropna(), reg['rescheduleDate'].dropna()]).index
reg = reg[~reg.index.isin(remove)]
reg

,gameGuid,link,gameType,season,gameDate,officialDate,isTie,gameNumber,publicFacing,doubleHeader,...,rescheduleDate,rescheduleGameDate,rescheduledFrom,rescheduledFromDate,resumeDate,resumeGameDate,resumedFrom,resumedFromDate,teams.away.team.placeholder,teams.home.team.placeholder
gamePk,,,,,,,,,,,,,,,,,,,,,
718780,28889d0d-b745-4054-b0d3-5fc4cfa06df9,/api/v1.1/game/718780/feed/live,R,2023,2023-03-30T17:05:00Z,2023-03-30,False,1,True,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
718781,ff222b1e-8305-4b37-89a5-2e150e1a708f,/api/v1.1/game/718781/feed/live,R,2023,2023-03-30T17:05:00Z,2023-03-30,False,1,True,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
718782,1f695ffe-1a00-41b0-b49e-49eba23de34b,/api/v1.1/game/718782/feed/live,R,2023,2023-03-30T18:10:00Z,2023-03-30,False,1,True,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
718777,5c968a1a-e61e-42fa-8609-720a021e6c35,/api/v1.1/game/718777/feed/live,R,2023,2023-03-30T18:20:00Z,2023-03-30,False,1,True,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
718776,a2bf6d6f-ec9e-44b3-9f37-9c4bfdc0969a,/api/v1.1/game/718776/feed/live,R,2023,2023-03-30T19:10:00Z,2023-03-30,False,1,True,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716355,5f6d16a1-ca5a-4499-accb-6b4bd5b83fff,/api/v1.1/game/716355/feed/live,R,2023,2023-10-01T19:10:00Z,2023-10-01,NaN,1,True,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716356,7cbab121-3edf-459a-bb68-e3ebd4d6ae87,/api/v1.1/game/716356/feed/live,R,2023,2023-10-01T19:10:00Z,2023-10-01,NaN,1,True,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716352,f4f42ab9-8d91-44e0-9fe5-4fa150005dac,/api/v1.1/game/716352/feed/live,R,2023,2023-10-01T19:10:00Z,2023-10-01,NaN,1,True,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
reg.columns.values

array(['gameGuid', 'link', 'gameType', 'season', 'gameDate',
       'officialDate', 'isTie', 'gameNumber', 'publicFacing',
       'doubleHeader', 'gamedayType', 'tiebreaker', 'calendarEventID',
       'seasonDisplay', 'dayNight', 'scheduledInnings',
       'reverseHomeAwayStatus', 'inningBreakLength', 'seriesDescription',
       'recordSource', 'ifNecessary', 'ifNecessaryDescription',
       'status.abstractGameState', 'status.codedGameState',
       'status.detailedState', 'status.statusCode', 'status.startTimeTBD',
       'status.abstractGameCode', 'teams.away.leagueRecord.wins',
       'teams.away.leagueRecord.losses', 'teams.away.leagueRecord.pct',
       'teams.away.score', 'teams.away.team.allStarStatus',
       'teams.away.team.id', 'teams.away.team.name',
       'teams.away.team.link', 'teams.away.team.season',
       'teams.away.team.venue.id', 'teams.away.team.venue.name',
       'teams.away.team.venue.link', 'teams.away.team.teamCode',
       'teams.away.team.fileCode', 'tea

In [7]:

played = reg.dropna(subset=['isTie']).copy() # filter to include only completed/current games
played_col_mapper = {'teams.home.team.abbreviation': 'team1', 'teams.away.team.abbreviation': 'team2', 
                     'teams.home.score': 'score1', 'teams.away.score': 'score2'}
played = played[played_col_mapper.keys()].rename(columns=played_col_mapper)
for col in ['score1', 'score2']:
    played[col] = played[col].astype(int)
played

,team1,team2,score1,score2
gamePk,,,,
718780,WSH,ATL,2,7
718781,NYY,SF,5,0
718782,BOS,BAL,9,10
718777,CHC,MIL,4,0
718776,TB,DET,4,0
...,...,...,...,...
716614,HOU,OAK,2,6
716609,COL,CHC,6,4
716606,SEA,LAA,8,0


In [8]:
remain_cols = {'teams.home.team.abbreviation': 'team1', 'teams.away.team.abbreviation': 'team2'}
remain = reg[~reg.index.isin(played.index)][remain_cols.keys()].rename(columns=remain_cols)
remain

,team1,team2
gamePk,,
716607,MIN,TB
716599,COL,CHC
716605,SF,CLE
716602,SEA,LAA
716600,PIT,WSH
...,...,...
716355,NYM,PHI
716356,CWS,SD
716352,KC,NYY


In [9]:
pd.Series(reg.index.isin(played.index)).value_counts()

True     2139
False     254
dtype: int64

In [10]:
reg.columns

Index(['gameGuid', 'link', 'gameType', 'season', 'gameDate', 'officialDate',
       'isTie', 'gameNumber', 'publicFacing', 'doubleHeader',
       ...
       'rescheduleDate', 'rescheduleGameDate', 'rescheduledFrom',
       'rescheduledFromDate', 'resumeDate', 'resumeGameDate', 'resumedFrom',
       'resumedFromDate', 'teams.away.team.placeholder',
       'teams.home.team.placeholder'],
      dtype='object', length=138)

In [11]:
all_gms['gameType'].value_counts()

R    2467
S     467
E      24
D      20
L      14
F      12
W       7
A       1
Name: gameType, dtype: int64

In [12]:
import datasource_mlb as ds

In [13]:
ds.get_ratings()

abbreviation
OAK    .317
PIT    .462
SD     .466
SEA    .552
SF     .510
STL    .441
TB     .610
TEX    .556
TOR    .552
MIN    .524
PHI    .545
ATL    .655
CWS    .386
MIA    .510
NYY    .503
MIL    .563
LAA    .466
AZ     .521
BAL    .632
BOS    .503
CHC    .534
CIN    .514
CLE    .473
COL    .361
DET    .458
HOU    .562
KC     .308
LAD    .611
WSH    .448
NYM    .458
Name: record.winningPercentage, dtype: object

In [14]:
ds.get_games()

(       team1 team2  score1  score2
 gamePk                            
 718780   WSH   ATL       2       7
 718781   NYY    SF       5       0
 718782   BOS   BAL       9      10
 718777   CHC   MIL       4       0
 718776    TB   DET       4       0
 ...      ...   ...     ...     ...
 716614   HOU   OAK       2       6
 716609   COL   CHC       6       4
 716606   SEA   LAA       8       0
 716619    SF   CLE       1       3
 716608   LAD    SD      11       2
 
 [2139 rows x 4 columns],
        team1 team2
 gamePk            
 716607   MIN    TB
 716599   COL   CHC
 716605    SF   CLE
 716602   SEA   LAA
 716600   PIT   WSH
 ...      ...   ...
 716355   NYM   PHI
 716356   CWS    SD
 716352    KC   NYY
 716364   MIL   CHC
 716353   STL   CIN
 
 [254 rows x 2 columns])

In [15]:
import season_simulator as ss

In [16]:
ss.compute_standings(ds.get_games()[0])

,W,L,wpct
team,,,
ATL,92,49,0.652482
BAL,89,53,0.626761
LAD,87,56,0.608392
TB,87,57,0.604167
HOU,82,63,0.565517
MIL,81,63,0.562500
TEX,80,64,0.555556
SEA,80,65,0.551724
PHI,77,63,0.550000
